# #310 Composite steel girders - Staged Construction
<i>Models a composite steel girder bridge with concrete deck slab poured in stages. Buckling analyses are performed for each intermediate construction stage</i>
***

In [ ]:
'''Inputs - Units are N,mm'''
 # Each segment of the girder has a different section, defined below
segments_lengths = [6000, 5000, 6000, 3000, 3000, 6000, 5000, 6000]
# Index in the array of segment lengths indicating the central support (two span bridge)
index_centre_support = 4 
mesh_size = 250

transverse_spacing = 4000 # Spacing of the two manin girders
deck_width = 7000         # Overall width of the concrete deck slab
deck_thk = 200            # Thickness of the concrete deck slab 

do_solve = False          # Carry out the analysis once the model is constructed
eigen_range = True        # Use a range for eigenvalue buckling analysis (avoids negative modes)

shear_stud_stiff = 1000   # Stiffness of the connection between girders and deck

# For each stage define the indicies of the segments to be activated
stage_activation ={1:[], 2:[0,1], 3:[2], 4:[3]}

# Loads
surf_load_intensity = -2.5
settlement = -0.01 # m
shrinkage_strain = -0.369764E-3 
construction_load = 10.0 # Construction load (N/mm) used in staged buckling analysis applied to the top flanges

Create a girder class to hold the section dimensions for each segment</br>
We'll create points in the LUSAS model for each section and then use the points to create surfaces joining them.

In [ ]:
class GirderSection:
    def __init__(self, tfb:float, tfthk:float, dw:float, tw:float, bfb:float, bfthk:float):
        self.top_flange_breadth = tfb
        self.top_flange_thk = tfthk
        self.web_depth = dw
        self.web_thk = tw
        self.bottom_flange_breadth = bfb
        self.bottom_flange_thk = bfthk
        # We'll save the points created in the model in these lists so we can later use them to define surfaces
        self.bf_points = []
        self.tf_points = []


Create a list of section definitions, we'll need one per segment + 1 for the end.

In [ ]:
sections1 : list[GirderSection] = []
sections1.append(GirderSection(500, 30, 1500, 15, 600, 40))
sections1.append(GirderSection(500, 50, 1500, 10, 600, 60))
sections1.append(GirderSection(500, 30, 1500, 15, 600, 40))
sections1.append(GirderSection(500, 30, 1500, 20, 600, 50))
sections1.append(GirderSection(500, 30, 1800, 20, 600, 50)) # Central support
sections1.append(GirderSection(500, 30, 1500, 15, 600, 40))
sections1.append(GirderSection(500, 50, 1500, 10, 600, 60))
sections1.append(GirderSection(500, 30, 1500, 15, 600, 40))
sections1.append(GirderSection(500, 30, 1500, 15, 600, 40))
# Check we have defined one more section then segment lengths
assert(len(segments_lengths) == len(sections1)-1)
import copy
sections2 = [copy.deepcopy(s) for s in sections1]

sections_list = [sections1, sections2]

Create a list of stiffener thicknesses that will be positioned at each change in section. A zero thickness indicates no stiffener.

In [ ]:
stiffener_thicknesses = [20, 10, 10, 20, 25, 20, 10, 10, 20]

# Ensure number of stiffener thicknesses matches the number of segments
assert(len(sections1) == len(stiffener_thicknesses))

#### Connect to LUSAS Modeller and create a new model

In [ ]:
import sys; sys.path.append('../') # Reference modules in parent directory
from LPI_22_0 import *
lusas = get_lusas_modeller()
if lusas.existsDatabase():
    raise Exception("This script will create a new model. Please save and close the current model and try again")

# Initialise the helpers module with a refernce to LUSAS
from m100_Tools_And_Helpers import Helpers
Helpers.initialise(lusas)

# Create a new model
lusas.newProject("Structural", "Two_Span_Composite_Bridge.mdl")
# Get a reference to the current model database for convenience
db = lusas.database() 
# 3D model with Z vertical
db.setAnalysisCategory("3D")
db.setVerticalDir("Z")
# Units N,mm
db.setModelUnits("N,mm,t,s,C")
# Consider only 5 degrees of freedom for thick shell elements
# This prevents diagonal decay of elements connected to the deck with joint elements in 3DOF
db.getOptions().setBoolean("Option 278", False, False, "")

In [ ]:
# When calling the LPI from an external process as we are here a speedup can be gained in v22 and later by disabling the UI
if lusas.getMajorVersionNumber() >=22:
    lusas.enableUI(False)

Create model attributes

In [ ]:
# Create the mesh attribute
shell_mesh_attr = db.createMeshSurface(f"Shell Mesh {mesh_size}mm").setRegularSize("QTS4", mesh_size, True)

# Bar mesh attributes
bracing_mesh_attr = db.createMeshLine("Bar Mesh").setNumber("BRS2", 1)

# Steel Material attribute
steel_material = db.createIsotropicMaterial("Steel", 200_000, 0.3, 7.8e-9, 10e-6)

# Concrete Material attribute
concrete_material           = db.createIsotropicMaterial("Concrete", 30_000, 0.2, 2.4e-9, 10e-6)
concrete_material_long_term = db.createIsotropicMaterial("Concrete (Long Term)", 30_000/3, 0.2, 2.4e-9, 10e-6)

# Bracing geometic attribute
bracing_section_attr = db.createGeometricLine("Bracing Section")
bracing_section_attr.setFromLibrary("UK Sections", "Equal Angles (Advance)", "150x150x12 UKEA", 0, 0, 0)
# Surface attributes will be created for each surface thickness below

# Connection between girders and deck is made with stiff joint elements.
# Create a joint mesh and material attribute for this connection
shear_stud_mesh_attr = db.createMeshLine("Shear Connection Mesh").setSize("JNT4", mesh_size)
shear_stud_material_attr = db.createSpringJointMaterial("Shear Connection Material", [shear_stud_stiff,shear_stud_stiff,shear_stud_stiff])
shear_stud_material_attr.setValue("Assignment", "Line")

# Support attributes
pinned = db.createSupportStructural("Fixed in XZ").setStructural("R", "F", "R", "F", "F", "F", "F", "F", "C", "F")
slide = db.createSupportStructural("Fixed in Z").setStructural("F", "F", "R", "F", "F", "F", "F", "F", "C", "F")

# Deactivate attribute disables elements to which its assigned
deactive_attr = db.createDeactivate("Deact1").setDeactivate("activeMesh", 100.0, 1.0E-6)
# Activate attribute reactivates attributes in the loadcase to which it's assigned
activate_attr = db.createActivate("Act1")

Create all the points for each section, saving the returned point in the Girder section definition. Use the helper fucntions library for create_point function

In [ ]:
s:GirderSection
for sections, y in [(sections1, -transverse_spacing/2),(sections2, transverse_spacing/2)]:
    x = 0
    for i in range(0, len(sections)):
        # Determine the longitudinal position (x coord) for each section
        if i > 0 : x += segments_lengths[i-1]
        # Determine the section to use at each longitudinal position
        s = sections[i]
        # Create the bottom flange points
        s.bf_points.append(Helpers.create_point(x, y + s.bottom_flange_breadth/2, -s.web_depth))
        s.bf_points.append(Helpers.create_point(x, y,                             -s.web_depth))
        s.bf_points.append(Helpers.create_point(x, y - s.bottom_flange_breadth/2, -s.web_depth))
        # Create the top flange points
        s.tf_points.append(Helpers.create_point(x, y +s.top_flange_breadth/2, 0.0))
        s.tf_points.append(Helpers.create_point(x, y,                         0.0))
        s.tf_points.append(Helpers.create_point(x, y -s.top_flange_breadth/2, 0.0))

Now join up the segment points. First create helper function to create surfaces and assign a geometric thickness attribute

In [ ]:
def create_surface(pnts: list, thk:float, ecc:float, group:str):

    geometry_data = lusas.geometryData().setAllDefaults()
    geometry_data.setLowerOrderGeometryType("points")

    obs = lusas.newObjectSet().add(pnts)

    surface = obs.createSurface(geometry_data).getObject("Surface")
    db.getGroupByName(group).add(surface, "Surfaces")

    name = f"{group} ({thk:.1f}mm)"

    db.createGeometricSurface(name).setSurface(thk, ecc).assignTo(surface)

Create groups to contain the girder surfaces

In [ ]:
db.createEmptyGroup("Top Flange")
db.createEmptyGroup("Web")
db.createEmptyGroup("Bottom Flange")
db.createEmptyGroup("Web Stiffeners")
db.createEmptyGroup("Deck")

Here we'll create the surfaces between each section definition using the points we created earlier

In [ ]:
for sections in sections_list:
    for i in range(0, len(segments_lengths)):

        s1:GirderSection = sections[i]   # Section definition at the start of the segment
        s2:GirderSection = sections[i+1] # Section definition at the end of the segment

        create_surface([ s1.bf_points[1], s2.bf_points[1], s2.bf_points[0], s1.bf_points[0] ], s1.bottom_flange_thk, 0.0, "Bottom Flange")
        create_surface([ s1.bf_points[2], s2.bf_points[2], s2.bf_points[0], s1.bf_points[0] ], s1.bottom_flange_thk, 0.0, "Bottom Flange")
        create_surface([ s1.bf_points[1], s2.bf_points[1], s2.tf_points[1], s1.tf_points[1] ], s1.web_thk,           0.0, "Web")
        create_surface([ s1.tf_points[1], s2.tf_points[1], s2.tf_points[0], s1.tf_points[0] ], s1.top_flange_thk,    0.0, "Top Flange")
        create_surface([ s1.tf_points[2], s2.tf_points[2], s2.tf_points[1], s1.tf_points[1] ], s1.top_flange_thk,    0.0, "Top Flange")


Create bearing stiffeners at the end sections

In [ ]:
for sections in sections_list:
    for i, thk in enumerate(stiffener_thicknesses):

        if thk > 0:

            s:GirderSection = sections[i]   # Section definition at the start of the segment
            create_surface([ s.bf_points[0], s.tf_points[0], s.tf_points[1], s.bf_points[1] ], thk, 0.0, "Web Stiffeners")
            create_surface([ s.bf_points[1], s.tf_points[1], s.tf_points[2], s.bf_points[2] ], thk, 0.0, "Web Stiffeners")


Assign a shell mesh attribute and steel material attribute to all surfaces

In [ ]:
# Assign mesh and steel material to all girder surfaces
assignment = lusas.assignment().setAllDefaults()
shell_mesh_attr.assignTo("Surfaces", assignment)
steel_material.assignTo("Surfaces", assignment)

In [ ]:
#assert False

Support attributes

In [ ]:
# To assign these support attributes we'll need to get hold of the lines. We can do this via the points that have created for each section

# Loop through each member
for sections in sections_list:
    # Assign supports to the end and central section
    for i, section in enumerate([sections[0], sections[index_centre_support], sections[-1]]):

        p0 = section.bf_points[0] # point at the +ve flange edge
        p1 = section.bf_points[1] # point at the centre of the web.
        p2 = section.bf_points[2] # point at the -ve flange edge

        # Loop through connected lines 
        for line in p1.getHOFs():
            line = win32.CastTo(line, "IFLine")
            # Points of the connected line
            ps1 = line.getStartPoint()
            ps2 = line.getEndPoint()
            # Check if the connected line points are those defining the bottom flange
            # If so then we can assign the relevant support attribute
            if ps1 == p0 or ps1 == p2 or ps2 == p0 or ps2 == p2:
                pinned.assignTo(line) if i == 1 else slide.assignTo(line)


In [ ]:
# Set Default Attributes for bracing members
db.setAsDefault("Mesh", bracing_mesh_attr)
db.setAsDefault("Material", steel_material)
db.setAsDefault("Line Geometric", bracing_section_attr)

In [ ]:
#bracing_lines = []
for i in range(0, len(segments_lengths)+1):
    line1 = Helpers.create_line_from_points(sections1[i].bf_points[1], sections2[i].tf_points[1])
    line2 = Helpers.create_line_from_points(sections1[i].tf_points[1], sections2[i].bf_points[1])
    #lusas.newObjectSet().add(line1).add(line2).intersectLines(lusas.geometryData().setAllDefaults())

    line3 = Helpers.create_line_from_points(sections1[i].bf_points[1], sections2[i].bf_points[1])
    line4 = Helpers.create_line_from_points(sections1[i].tf_points[1], sections2[i].tf_points[1])

In [ ]:
# Clear default attribute assignments
db.setAsDefault("Mesh", "None")
db.setAsDefault("Material", "None")
db.setAsDefault("Line Geometric", "None")

# Slab Deck

In [ ]:
x_grid = [0]
for s in segments_lengths:
    x_grid.append(x_grid[-1]+s)

y_grid = [-deck_width/2, -transverse_spacing/2, 0.0, transverse_spacing/2, deck_width/2]

In [ ]:
# Create the actual points in the model
deck_points = [[Helpers.create_point(x, y, deck_thk/2) for x in x_grid] for y in y_grid]
# [Y][X]

# Create the surfaces for the deck
for iy in range(0, len(y_grid)-1):
    for ix in range(0, len(x_grid)-1):
        create_surface([ deck_points[iy][ix], deck_points[iy][ix+1], deck_points[iy+1][ix+1], deck_points[iy+1][ix] ], deck_thk, 0.0, "Deck")

In [ ]:
shell_mesh_attr.assignTo("Group", "Deck")
concrete_material.assignTo("Group", "Deck")

# Deck to Girder connection

In [ ]:
# Helper function to find the lines in the top flanges and deck surfaces
def get_lines_x(points:list[IFPoint])->list[IFLine]:
    lines = []
    for point in points:
        for hof in point.getHOFs():
            if hof.getTypeCode() == 2 : # Line
                # Check it is aligned with and parallel to the x axis
                if abs(hof.getEndPosition()[1] - hof.getStartPosition()[1]) < 1e-3 and \
                   abs(hof.getEndPosition()[2] - hof.getStartPosition()[2]) < 1e-3:
                    # Add to the list if not already in there
                    if not hof in lines:
                        lines.append(hof)
    return lines

# List of lines in section 1 top flange
lines_s1_tf   = get_lines_x([s.tf_points[1] for s in sections1])
# List of lines in deck corresponding to the lines in section 1
lines_s1_deck = get_lines_x([p for p in deck_points[1]])

# List of lines in section 1 top flange
lines_s2_tf   = get_lines_x([s.tf_points[1] for s in sections2])
# List of lines in deck corresponding to the lines in section 2
lines_s2_deck = get_lines_x([p for p in deck_points[-2]])


# Default assignment
assignment = lusas.assignment().setAllDefaults()

# Assign the mesh between corresponding lines of the girder top flange and deck
for i in range(0, len(lines_s1_tf)):
    objs = lusas.newObjectSet().add(lines_s1_deck[i]).add(lines_s1_tf[i])
    shear_stud_mesh_attr.assignTo(objs,assignment)
    del objs

# Assign the mesh between corresponding lines of the girder top flange and deck
for i in range(0, len(lines_s2_tf)):
    objs = lusas.newObjectSet().add(lines_s2_deck[i]).add(lines_s2_tf[i])
    shear_stud_mesh_attr.assignTo(objs,assignment)
    del objs

# Assign the material for the joint elements to the primary line of the joint assignment
shear_stud_material_attr.assignTo(lines_s1_deck, assignment)
shear_stud_material_attr.assignTo(lines_s2_deck, assignment)

# Loading

In [ ]:
# Get the automatically created loadcase in analysis 1 and add automatic gravity to it
# The loadcase will be used only to test the model is working correctly
# NOTE: getLoadset and setName funcction returns a reference to the IFLoadset baseclass and must be cast to IFLoadcase to access the addGravity function
win32.CastTo(db.getLoadset("Loadcase 1", 0).setName("Gravity"), "IFLoadcase").addGravity(True)
db.getAnalysis("Analysis 1").setName("00 Base")

In [ ]:
# Create 4 separate analyses, 1st phase is for the girders only, 2nd phase is long term concrete 3rd phases is short term concrete, and the 4th analysis is the construction stages
phase1_analysis = db.createAnalysisStructural("Phase 1", False)
phase2_analysis = db.createAnalysisStructural("Phase 2", False)
phase3_analysis = db.createAnalysisStructural("Phase 3", False)
constr_analysis = db.createAnalysisStructural("Phase 4 - Construction Sequence", False)

### Phase1 (Girders only) loading

In [ ]:
# Phase 1 - Steel Only - Self weight loadcases
steel_sw_loadcase = db.createLoadcase("Steel Self Weight", phase1_analysis.getName())
steel_sw_loadcase.addGravity(True)


wet_conc_loadcase = db.createLoadcase("Wet Concrete", phase1_analysis.getName())

### Phase 2 (long term concrete) loading

In [ ]:
# Phase 2 - Long term - Super imposed loadcase
sid_loadcase = db.createLoadcase("Surfacing", phase2_analysis.getName())

# Assign long term concrete material to the deck surfaces
assignment = lusas.assignment().setAllDefaults().setLoadset(sid_loadcase)
concrete_material_long_term.assignTo("Group", "Deck", assignment)


# Create a locally distributed load for Surfacing loads
surf_load_attr = db.createLoadingLocalDistributed("Surfacing").setLocalDistributed(0.0, 0.0, surf_load_intensity, "surface")
# and assign to all the surfaces
surf_load_attr.assignTo("Surfaces", assignment)


In [ ]:
shrink_loadcase = db.createLoadcase("Shrinkage", phase2_analysis.getName())

# Create an initial stress/strain attribute and populate only the strains Ex,Ey at the top middle and bottom of the shell
# The indicies of these components are specified in the list, all other values are zero
strain_array = [0]*30
for i in [5,6,15,16,25,26]:
    strain_array[i] = shrinkage_strain

# Create the attribute
shrink_attr = db.createLoadingStressStrain("Shrinkage strain").setStressStrain("Initial", strain_array, "Surface", "Thick shell", 3)
# Set the loadcase in the assignment object
assignment = lusas.assignment().setAllDefaults().setLoadset(shrink_loadcase)
# Use the surface set defined early to assign the load to all deck surfaces
shrink_attr.assignTo("Group", "Deck", assignment)

In [ ]:
# Settlement loadcases
settlement_loadcases = [db.createLoadcase(f"Settlement {i}", phase2_analysis.getName()) for i in range(1, 4)]

# Create the prescribed displacement attribute to represent vertical settlement
settlement_attr = db.createPrescribedDisplacementLoad("Settlement", "Total")
settlement_attr.setDisplacement("W", settlement)

# Assign the settlement loads at each support
# Loop through each member
for sections in sections_list:
    # Supports are at the end and central sections
    for i, section in enumerate([sections[0], sections[index_centre_support], sections[-1]]):

        p0 = section.bf_points[0] # point at the +ve flange edge
        p1 = section.bf_points[1] # point at the centre of the web.
        p2 = section.bf_points[2] # point at the -ve flange edge

        assignment = lusas.assignment().setAllDefaults().setLoadset(settlement_loadcases[i])

        # Loop through connected lines 
        for line in p1.getHOFs():
            line = win32.CastTo(line, "IFLine")
            # Points of the connected line
            ps1 = line.getStartPoint()
            ps2 = line.getEndPoint()
            # Check if the connected line points are those defining the bottom flange
            # If so then we can assign the relevant settlment attribute
            if ps1 == p0 or ps1 == p2 or ps2 == p0 or ps2 == p2:
                settlement_attr.assignTo(line, assignment)

### Phase 3 (short term concrete) loading

In [ ]:
# Phase 3 - Short term
t1_loadcase = db.createLoadcase("Temperature Increase", phase3_analysis.getName())
t2_loadcase = db.createLoadcase("Temperature Decrease", phase3_analysis.getName())
t11_loadcase = db.createLoadcase("Temperature Daily Warm", phase3_analysis.getName())
t21_loadcase = db.createLoadcase("Temperature Daily Cool", phase3_analysis.getName())

# Wind loadcases
wind_loadcases = [db.createLoadcase(f"Wind {i}", phase3_analysis.getName()) for i in range(1, 4)]

# Traffic breaking loadcases
braking1_loadcase = db.createLoadcase("Braking 1", phase3_analysis.getName())
braking2_loadcase = db.createLoadcase("Braking 2", phase3_analysis.getName())

<div class="alert alert-block alert-info">
<b>Note:</b> Traffic live loads should be generated with the traffic load optimiser, this is not considered as part of the example</div>

### Staged construction buckling analysis

In [ ]:
# Construction load attribute
construction_load_attr = db.createLoadingGlobalDistributed("Global Distributed Load")
construction_load_attr.setGlobalDistributed("Length", 0.0, 0.0, -construction_load, 0.0, 0.0, 0.0, 0.0, 0.0, False, 0.0)

In [ ]:

# Create loadcases for each stage of construction
stage_loadcases = []
for i in range(1, 5):
    loadcase = db.createLoadcase(f"Stage {i}", constr_analysis.getName())
    loadcase.addGravity(True)
    if i == 1:
        # Set the default nonlinear control for the first loadcase, this is inherited by the subsequent loadcases
        loadcase.setTransientControl(0)
        loadcase.getTransientControl().setNonlinearManual().setOutput()
    stage_loadcases.append(loadcase)
    

# For each construction stage create a branched analysis to consider buckling at each stage of construction
for i, stage in enumerate(stage_loadcases):
    branch = db.createAnalysisBranch(stage, f"Stage {i+1} Stability", False, "Eigen analysis")
    loadcase = db.createLoadcase(f"Stage {i+1} Buckling", branch.getName())
    loadcase.addGravity(True)
    if eigen_range:
        # Set the loadcase controls to be a Buckling analysis searching for 5 bucling modes within the given range of load factors
        loadcase.setEigenvalueRangeControl("Buckling", "Buckling load", 10, 0.0, 5)
    else:
        # Set the loadcase controls to be a Buckling analysis searching for the minimum 5 buckling modes
        loadcase.setEigenvalueMaxMinControl("Buckling", "Minimum", 5)

    loadcase.getEigenvalueControl().setValue("Eigensolver", "Default")

    construction_load_attr.assignTo(lines_s1_tf, lusas.assignment().setAllDefaults().setLoadset(loadcase))
    construction_load_attr.assignTo(lines_s2_tf, lusas.assignment().setAllDefaults().setLoadset(loadcase))

In [ ]:
# Deactivate the deck in the start loadcase in the staged construction analysis
for loadcase in [steel_sw_loadcase, stage_loadcases[0]]:
    assignment = lusas.assignment().setAllDefaults().setLoadset(loadcase)
    deactive_attr.assignTo("Group", "Deck", assignment)
    deactive_attr.assignTo(lines_s1_deck, assignment)
    deactive_attr.assignTo(lines_s2_deck, assignment)

In [ ]:
# Activate the parts of the deck that correspond to each stage of construction
for i in range(1, len(stage_loadcases)+1):
    loadcase = stage_loadcases[i-1]
    # Activate the parts of the deck corresponding to the stage
    if i in stage_activation and len(stage_activation[i]) > 0:
        objs = lusas.newObjectSet()
        for seg in stage_activation[i]:
            objs.add(lines_s1_deck[seg])
            objs.add(lines_s1_deck[-1-seg])
            objs.add(lines_s2_deck[seg])
            objs.add(lines_s2_deck[-1-seg])
        objs.addHOF()
        activate_attr.assignTo(objs, lusas.assignment().setAllDefaults().setLoadset(loadcase))

Set the model to display the assigned plate thicknesses and supports in an isometric orientation

In [ ]:
lusas.view().insertGeometryLayer() # Make sure the drawing layers exist before accessing them
lusas.view().insertAttributesLayer()

lusas.view().geometry().autoColourByAttributes("Geometric", True)
lusas.view().attributes().visualiseAll("Supports")
lusas.view().setIsometric()

In [ ]:
# Mesh the model
db.resetMesh()
db.updateMesh()

In [ ]:
# Solve all the anlyses
if do_solve:
    db.getAnalysis("Analysis 1").solve(True)
    db.getAnalysis("Eigenvalue Buckling Analysis").solve(True)
    db.openAllResults(False)


In [ ]:
# Holding onto references to LUSAS objects can cause issues if they are then manually edited or deleted.
# Here we remove all those references
for v in vars():
    del v

In [ ]:
lusas.enableUI(True)